<a href="https://colab.research.google.com/github/pepperamy/disclosure_change/blob/main/test_change_difference_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

import sys, os
cur_path = os.path.join('/content/drive/My Drive/change','data')
os.chdir(cur_path)
print(os.getcwd())

Mounted at /content/drive
/content/drive/My Drive/change/data


In [ ]:
import random, pickle
import numpy as np
from torch.nn import BCEWithLogitsLoss, BCELoss, MSELoss
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix, f1_score, accuracy_score, precision_recall_fscore_support
import tensorflow as tf
import torch
import pandas as pd

!pip install transformers
from transformers import AutoConfig, AutoModel, AutoTokenizer, AutoModelForSequenceClassification

!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, util

import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

!pip install torchinfo
from torchinfo import summary
from sklearn.model_selection import train_test_split, StratifiedKFold

import copy
from sklearn.utils import shuffle
import glob

import time
import datetime

from itertools import cycle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.0 MB/s 
     |████████████████████████████████| 182 kB 61.4 MB/s 
     |████████████████████████████████| 7.6 MB 60.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.0 MB/s 
     |████████████████████████████████| 1.3 MB 9.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=a8c6753f6501d7d726ddbe99c2d12055b7ae9f57537c41d4613e530b32915776
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, 
and then re-execute this cell.


In [ ]:
# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    device_name ="/cpu:0"
    print('GPU device not found')
    #raise SystemError('GPU device not found')

GPU device not found


In [ ]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [ ]:
para_map = pickle.load(open("./paragraphs_1994_2016-001.pkl", 'rb'))
#para_map = pickle.load(open("data/mda/paragraphs_1994_2016_original.pkl", 'rb'))
len(para_map)

20893

In [ ]:
def get_df_for_cik(cik):
    
    df = pd.concat({k: pd.Series(v) for k, v in para_map[cik].items()})
    df = df.reset_index()
    
    df.columns = ['fyear','pid','text']
    df = df.sort_values(by=['fyear','pid'])
    df = df.reset_index(drop = True)
    
    return df

In [ ]:
cik = 1007021

df = get_df_for_cik(cik)
df.head()
df.groupby("fyear")["pid"].count()

,fyear,pid,text
0,1996,0,The following discussion and analysis of the f...
1,1996,1,The Company does not provide forecasts of the ...
2,1996,2,The software industry is highly competitive an...
3,1996,3,Revenue. The Company's revenue consists of lic...
4,1996,4,International revenue decreased dd% to $dd in ...


fyear
1996    26
1997    28
1998    36
1999    35
2000    37
2001    37
Name: pid, dtype: int64

In [ ]:
df[df.fyear == 1996]

,fyear,pid,text
0,1996,0,The following discussion and analysis of the f...
1,1996,1,The Company does not provide forecasts of the ...
2,1996,2,The software industry is highly competitive an...
3,1996,3,Revenue. The Company's revenue consists of lic...
4,1996,4,International revenue decreased dd% to $dd in ...
5,1996,5,Software license revenue decreased dd% in fisc...
6,1996,6,Maintenance and services revenue increased dd%...
7,1996,7,Cost of Revenue and Gross Profit. The Company'...
8,1996,8,Cost of maintenance and services revenue consi...
9,1996,9,Selling and Marketing. Selling and marketing e...


In [ ]:
df[df.fyear == 1997]

,fyear,pid,text
26,1997,0,The following discussion and analysis of the f...
27,1997,1,Revenue. The Company's revenue consists of lic...
28,1997,2,Software license revenue decreased dd% in fisc...
29,1997,3,Maintenance and services revenue increased dd%...
30,1997,4,Cost of Revenue and Gross Profit. The Company'...
31,1997,5,Cost of maintenance and services revenue consi...
32,1997,6,Selling and Marketing. The Company's products ...
33,1997,7,Research and Development. Research and develop...
34,1997,8,"General and Administrative. In fiscal dd, gene..."
35,1997,9,Interest Income. Interest income consists prim...


# test sbert

In [ ]:
sbert_model = SentenceTransformer('all-mpnet-base-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

## test similarity by sentence

In [ ]:
s1 = df[df.fyear == 1996].text[0]
s1

'The following discussion and analysis of the financial condition and results of operations of the Company should be read in conjunction with the financial statements and related notes thereto.'

In [ ]:
s2 = df[df.fyear == 1997].text[26]
s2

'The following discussion and analysis of the financial condition and results of operations of the Company should be read in conjunction with the financial statements and related notes thereto.'

In [ ]:
s3 = df[df.fyear == 1996].text[3]
s3

"Revenue. The Company's revenue consists of license fees for its CMS family of product data management software products and fees for professional services and software maintenance. The Company had total revenue of $dd in fiscal dd as compared to total revenue of $dd in fiscal dd, a decrease of less than dd%."

In [ ]:
s4 = df[df.fyear == 1997].text[27]
s4

"Revenue. The Company's revenue consists of license fees for its CMS family of product data management software products and fees for professional services and software maintenance. The Company had total revenue of $dd in fiscal dd as compared to total revenue of $dd in fiscal dd, a decrease of $dd or dd%."

In [ ]:
# s5 is the modification of s4 
s5 = '''
   Revenue. The Company's revenue consists of license fees for its CMS house of product data management software products and fees for professional services and  maintenance.
  The Company had total revenue of $dd in fiscal dd.
  '''

In [ ]:
embeddings1 = sbert_model.encode(s1, convert_to_tensor=True)
embeddings2 = sbert_model.encode(s2, convert_to_tensor=True)
embeddings3 = sbert_model.encode(s3, convert_to_tensor=True)
embeddings4 = sbert_model.encode(s4, convert_to_tensor=True)
embeddings5 = sbert_model.encode(s5, convert_to_tensor=True)

In [ ]:
# s1 == s2
cosine_scores = util.cos_sim(embeddings1, embeddings2)
cosine_scores

tensor([[1.0000]])

In [ ]:
# s3 ~= s4
cosine_scores = util.cos_sim(embeddings3, embeddings4)
cosine_scores

tensor([[0.9949]])

In [ ]:
# s1 != s3
cosine_scores = util.cos_sim(embeddings1, embeddings3)
cosine_scores

tensor([[0.3781]])

In [ ]:
# s5 is the modification of s4 
cosine_scores = util.cos_sim(embeddings5, embeddings4)
cosine_scores

tensor([[0.9125]])

## test similarity by whole doc

In [ ]:
d1 = df[df.fyear == 1996].text

In [ ]:
d2 = df[df.fyear == 1997].text

In [ ]:
d1_emb = sbert_model.encode(d1.values, convert_to_tensor=True)
d2_emb = sbert_model.encode(d2.values, convert_to_tensor=True)

In [ ]:
d1_emb.shape

In [ ]:
# pad d1_emb, so the size of d1 == d2
import torch.nn.functional as F
d1_emb = F.pad(input=d1_emb, pad=(0,0,0,2), mode='constant', value=0)
# data = d1_emb
# # pad(left, right, top, bottom)
# new_data = F.pad(input=data, pad=(0,0,0,2), mode='constant', value=0)
# print(new_data.shape)
print(d1_emb.shape)

In [ ]:
d2_emb.shape

In [ ]:
cosine_scores = util.cos_sim(d2_emb, d1_emb)

In [ ]:
cosine_scores.shape

In [ ]:
cosine_scores

# test bert

In [ ]:
# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
# Print the original sentence.
print(' Original: ', s1)

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(s1))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(s1)))

In [ ]:

bert_model = AutoModel.from_pretrained(
    # 'ProsusAI/finbert',
    'bert-base-uncased',
    # 'yiyanghkust/finbert-pretrain',
    num_labels = 2, 
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = True, # Whether the model returns all hidden-states.
    )
bert_model.cuda()

In [ ]:
# Put everything together as a function. This is for pretrained word vectors

def get_pretrained_wordvector(sentences, tokenizer, bert_model):

    input_ids = []
    attention_masks = []
    max_len = 100

    # For every sentence...
    for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
        encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = max_len,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        #padding='max_length',
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
        # Add the encoded sentence to the list.    
        input_ids.append(encoded_dict['input_ids'])
    
        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks.append(encoded_dict['attention_mask'])


    # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    bert_model.eval()
    with torch.no_grad():
        outputs = bert_model(input_ids.to(device), attention_masks.to(device))   
        hidden_states = outputs[2]

    
    # get the last four layers
    token_embeddings = torch.stack(hidden_states[-4:], dim=0) 
    #print(token_embeddings.size())

    # permute axis
    token_embeddings = token_embeddings.permute(1,2,0,3)
    #print(token_embeddings.size())

    # take the mean of the last 4 layers
    token_embeddings = token_embeddings.mean(axis=2)

    #print(token_embeddings.size())

    return token_embeddings, attention_masks

In [ ]:
d1_tkn_emb, d1_att_msk = get_pretrained_wordvector(d1.values, tokenizer, bert_model)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
d2_tkn_emb, d2_att_msk = get_pretrained_wordvector(d2.values, tokenizer, bert_model)

In [ ]:
# expand the shape of d1, so that d1 == d2
print(d1_tkn_emb.shape, d1_att_msk.shape)
print(d2_tkn_emb.shape, d2_att_msk.shape)

torch.Size([26, 100, 768]) torch.Size([26, 100])
torch.Size([28, 100, 768]) torch.Size([28, 100])


In [ ]:
d1_tkn_emb = F.pad(input=d1_tkn_emb, pad=(0,0,0,0,0,2), mode='constant', value=0)
d1_att_msk = F.pad(input=d1_att_msk, pad=(0,0,0,2), mode='constant', value=0)
print(d1_tkn_emb.shape, d1_att_msk.shape)

torch.Size([30, 100, 768]) torch.Size([30, 100])


In [ ]:
# define Siamese model
class simple_siamese(nn.Module):
    def __init__(self, emb_dim, wrd_len, num_filters, kernel_sizes, dropout_rate):
        super.__init__()
        self.emb_dim = emb_dim
        self.wrd_len = wrd_len
        self.num_filters = num_filters
        self.kernel_sizes = kernel_sizes
        self.dropout_rate = dropout_rate
        


        self.conv = nn.Sequential(
            nn.Conv2d(768, 256, kernal_size = kernel_sizes1), # input (#batch, 768, num_para->60, num_words->100) # kernal size = 10,50  # output: (#batch, 256, 50, 50)
            nn.ReLU(inplace=True),
            nn.MaxPool2d(5, padding=0),  # input (#batch, 256, 50, 50) #output (#batch, 256, 10, 10)
            nn.Conv2d(256, 128, kernal_size = kernel_sizes2) # input (#batch, 256, num_para->10, num_words->10) # kernal size = 3,3
            nn.ReLU(inplace=True),
            nn.MaxPool2d(7, padding=0),
            nn.ReLU(), 
        )

        self.fc = nn.Linear(7*7*128, self.num_classes)
        self.dropout = nn.Dropout(p=dropout_rate)
        

    def forward(self, input1, input2):
        #permute input to make it fit cnn
        x1 = torch.permute(input1, (0,3,1,2))
        x2 = torch.permute(input1, (0,3,1,2))
        print(x1.size())
        print(x2,size())

        x1 = conv(input1)
        x2 = conv(input2)
        x = torch.sub(x1,x2)

        x = self.dropout(x)
        logit = self.fc(x)

        return logit

In [ ]:
layer = nn.Conv2d(7,2,kernel_size=3,stride=1,padding=0)
layer = layer.to(memory_format=torch.channels_last)

In [ ]:
input = torch.rand(1,3,7,7)

In [ ]:
input = input.to(memory_format=torch.channels_last)

In [ ]:
input

tensor([[[[0.9770, 0.9942, 0.8447, 0.7949, 0.5914, 0.5622, 0.9080],
          [0.4687, 0.8073, 0.2108, 0.3913, 0.3690, 0.6596, 0.9614],
          [0.1283, 0.1448, 0.1826, 0.9285, 0.9334, 0.3711, 0.2762],
          [0.0481, 0.4341, 0.2451, 0.0496, 0.9781, 0.1335, 0.6688],
          [0.8188, 0.4419, 0.3134, 0.1130, 0.8135, 0.0042, 0.9304],
          [0.5185, 0.2473, 0.4666, 0.6786, 0.1495, 0.9492, 0.6200],
          [0.4047, 0.5539, 0.0594, 0.7656, 0.2328, 0.9196, 0.7855]],

         [[0.9290, 0.9034, 0.7188, 0.5566, 0.2085, 0.9788, 0.4177],
          [0.1740, 0.3691, 0.6150, 0.6238, 0.2789, 0.7051, 0.2802],
          [0.9412, 0.8973, 0.8211, 0.7401, 0.4112, 0.1328, 0.8265],
          [0.5515, 0.6313, 0.4303, 0.8721, 0.3737, 0.6719, 0.0552],
          [0.6573, 0.6397, 0.3129, 0.9325, 0.5548, 0.0563, 0.0749],
          [0.3275, 0.0599, 0.3578, 0.6379, 0.8480, 0.2638, 0.4767],
          [0.6068, 0.4065, 0.9386, 0.4172, 0.6754, 0.9635, 0.5884]],

         [[0.3198, 0.3294, 0.0790, 0.9723, 0

In [ ]:
input.shape

torch.Size([1, 3, 7, 7])

In [ ]:
out = layer.forward(input)

RuntimeError: ignored

In [ ]:
out.shape

torch.Size([1, 2, 5, 5])

In [ ]:
out.shape

torch.Size([1, 2, 5, 5])

In [ ]:
input = torch.randn(1, 10, 32, 32)
model = torch.nn.Conv2d(10, 20, 1, 1)
output = model(input)

In [ ]:
output

tensor([[[[-1.5246,  0.0969, -0.4643,  ..., -0.8161,  0.2054, -0.4162],
          [-0.3757,  0.0591,  0.1835,  ...,  0.5914,  0.5575,  0.2323],
          [ 1.3308,  0.2941, -0.7461,  ...,  0.4737, -0.2250, -0.0541],
          ...,
          [ 0.1721,  0.2945,  0.1694,  ..., -1.2478, -0.0751,  0.6802],
          [-0.2528, -0.5959,  0.5163,  ..., -0.4380, -0.2068, -0.4798],
          [ 0.2973, -0.2572,  0.5566,  ..., -0.0246,  0.3292, -0.3902]],

         [[ 0.2161,  0.2108, -1.2358,  ..., -0.1750,  0.2825, -0.0434],
          [-0.7063,  0.6281,  0.2803,  ..., -0.2108,  1.3149,  0.5091],
          [ 0.2837, -0.0526, -0.0173,  ..., -0.7126,  0.2507,  0.9840],
          ...,
          [ 0.3238,  0.9016,  0.1037,  ..., -0.2346,  0.0731,  0.9006],
          [ 0.4946, -0.0902,  0.3274,  ..., -0.6771,  1.1469, -1.0037],
          [-0.5976, -0.4670, -0.0387,  ...,  0.1788,  0.5447,  0.9850]],

         [[ 0.1994,  0.6606, -0.3578,  ...,  0.4490,  0.1219,  0.8431],
          [ 1.0490, -0.3250,  

In [ ]:
output.shape

torch.Size([1, 20, 32, 32])

In [ ]:
input = torch.randn(1, 10, 32, 32)
model = torch.nn.Conv2d(10, 20, 1, 1)
### NB: convert to Channels Last memory format.
###   oneDNN support NHWC for feature maps (input, output),
###   but weight still need to be of blocked format.
###   Still we can save reorders for feature maps.
input = input.to(memory_format=torch.channels_last)
model = model.to(memory_format=torch.channels_last)
output = model(input)

In [ ]:
output.shape

torch.Size([1, 20, 32, 32])

In [ ]:
output

tensor([[[[-0.4064, -0.3341, -0.4267,  ..., -0.1119,  0.2019, -0.1108],
          [-0.1547, -1.3274, -0.4226,  ..., -0.6515,  0.0466,  0.3196],
          [ 0.2739,  0.3717, -0.0519,  ..., -0.0942,  0.1112, -0.3713],
          ...,
          [-0.0331, -0.6668,  0.2002,  ...,  0.9945, -0.4735,  0.7592],
          [ 0.0366,  0.4841,  0.6297,  ...,  1.2383,  0.5363,  0.1422],
          [-0.1117,  1.0904,  0.2876,  ...,  0.0058, -1.0127,  0.1612]],

         [[ 1.5293,  0.5726, -0.6352,  ...,  0.1075,  0.1770, -1.1247],
          [ 0.1985, -0.8593, -0.1028,  ...,  0.2187, -0.0643, -1.0390],
          [ 0.2129, -0.7318, -0.4826,  ...,  0.2636,  0.5027, -0.4365],
          ...,
          [ 0.7671, -0.6979, -0.0922,  ..., -1.1099, -0.3734, -0.3994],
          [ 0.1774, -0.5995, -0.0775,  ..., -0.7425, -0.6735, -0.4673],
          [-0.6867, -0.4084,  0.2771,  ..., -0.5984, -0.7740,  0.4460]],

         [[-0.6558, -0.4210, -0.4707,  ..., -0.5991, -0.2380,  0.2801],
          [ 0.4981, -1.5274, -